In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential, Model
from keras.layers import Dense, Input
np.set_printoptions(precision=3, suppress=True)

from collect_data import vahid_data, amanda_data, manuel_data

2024-06-03 23:12:21.102180: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-03 23:12:21.126599: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
input_dim = 3

encoders = []
decoders = []
autoencoders = []

input_layer = Input(shape=(input_dim,))

In [3]:
for i in range(216):
    encoder = Dense(32, activation='silu')(input_layer)
    encoder = Dense(1, activation='silu')(encoder)  # Bottleneck layer (1D representation)
    encoders.append(encoder)

    # Define the decoder
    decoder = Dense(32, activation='silu')(encoder)
    decoder = Dense(input_dim, activation='linear')(decoder)
    decoders.append(decoder)

    # Define the autoencoder model
    autoencoder = Model(inputs=input_layer, outputs=decoder)
    autoencoders.append(autoencoder)

    # Compile the model
    autoencoders[i].compile(optimizer='adam', loss='mean_squared_error')

2024-06-03 23:12:22.038164: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-03 23:12:22.062378: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-03 23:12:22.062500: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
for i in range(6):
    for j in range(12):
       autoencoders[12*i+j].fit(vahid_data[i][j], vahid_data[i][j], epochs=100, verbose=0)
       autoencoders[12*6+12*i+j].fit(amanda_data[i][j], amanda_data[i][j], epochs=100, verbose=0)
       autoencoders[12*6*2+12*i+j].fit(manuel_data[i][j], manuel_data[i][j], epochs=100, verbose=0)
    print(f"All sets completed for all category {i+1}.")

I0000 00:00:1717477945.030395 1720541 service.cc:145] XLA service 0x7bf500004580 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1717477945.030427 1720541 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-06-03 23:12:25.042902: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-03 23:12:25.101593: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1717477945.379541 1720541 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


All sets completed for all category 1.
All sets completed for all category 2.
All sets completed for all category 3.
All sets completed for all category 4.
All sets completed for all category 5.
All sets completed for all category 6.


In [8]:
encoder_models = []
for i in range(216):
    encoder_models.append(Model(inputs=input_layer, outputs=encoders[i]))

In [9]:
encoder_models[71].predict(vahid_data[5][11])

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


array([[49.54    ],
       [49.672375],
       [50.766495],
       [50.588665],
       [49.924377],
       [49.73736 ],
       [49.69736 ],
       [50.03172 ],
       [50.47773 ],
       [50.68035 ],
       [50.64939 ],
       [50.584763],
       [50.65768 ],
       [50.845894],
       [50.944817],
       [51.406532],
       [51.48332 ],
       [51.67856 ],
       [51.69997 ],
       [51.98721 ],
       [52.29439 ],
       [52.463768],
       [52.695026],
       [52.738808],
       [53.045998],
       [53.235756],
       [53.296047],
       [53.542442],
       [53.508377],
       [53.594124],
       [53.79476 ],
       [53.83609 ],
       [53.99087 ],
       [54.108505],
       [54.039043],
       [54.226105],
       [54.16047 ],
       [54.01319 ],
       [54.27841 ],
       [54.28989 ],
       [54.43647 ],
       [54.541267],
       [54.67325 ],
       [55.079723],
       [55.367523],
       [55.60569 ],
       [55.771515],
       [55.854538],
       [56.332077],
       [56.5516  ],
